# Overfit Guard - Universal Dataset Demo

**Run Overfit Guard on ANY dataset - Built-in or Custom!**

This notebook lets you:
- Choose from built-in datasets (Classification & Regression)
- Upload your own CSV dataset
- Select framework (PyTorch, Keras, or scikit-learn)
- Compare training WITH vs WITHOUT overfit protection
- See automatic overfitting detection and correction in action

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Core-Creates/overfit-guard/blob/main/notebooks/universal_dataset_demo.ipynb)

---

## 1. Installation

Install Overfit Guard and dependencies:

In [ ]:
%%capture
# Install from GitHub repo (always gets latest version)
!pip install git+https://github.com/Core-Creates/overfit-guard.git

# Install ML frameworks
!pip install scikit-learn pandas matplotlib seaborn torch torchvision tensorflow

print("✅ Installation complete!")

## 2. Configuration - Choose Your Setup

**Customize these settings:**

In [ ]:
# ========== CONFIGURATION ==========

# DATASET SELECTION
# Built-in options: 'breast_cancer', 'digits', 'wine', 'iris', 'diabetes', 'california_housing', 'custom'
DATASET = 'breast_cancer'  # Change this!

# FRAMEWORK SELECTION
# Options: 'pytorch', 'keras', 'sklearn'
FRAMEWORK = 'pytorch'  # Change this!

# TRAINING SETTINGS
NUM_EPOCHS = 100
BATCH_SIZE = 32
LEARNING_RATE = 0.001

# CUSTOM DATASET (only used if DATASET='custom')
CUSTOM_CSV_PATH = None  # Set to your CSV file path
TARGET_COLUMN = 'target'  # Name of target column in CSV

# ===================================

print(f"✓ Dataset: {DATASET}")
print(f"✓ Framework: {FRAMEWORK}")
print(f"✓ Epochs: {NUM_EPOCHS}")

## 3. Load Dataset

Load your chosen dataset (or upload custom):

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def load_dataset(dataset_name, custom_path=None, target_col='target'):
    """Load dataset and return X, y, task_type, num_classes."""
    
    if dataset_name == 'custom':
        # Load custom CSV
        if custom_path is None:
            # Try to upload in Colab
            try:
                from google.colab import files
                uploaded = files.upload()
                custom_path = list(uploaded.keys())[0]
            except:
                raise ValueError("Please provide CUSTOM_CSV_PATH or upload a file")
        
        df = pd.read_csv(custom_path)
        X = df.drop(columns=[target_col]).values
        y = df[target_col].values
        
        # Detect task type
        num_unique = len(np.unique(y))
        if num_unique <= 20:
            task_type = 'classification'
            num_classes = num_unique
        else:
            task_type = 'regression'
            num_classes = 1
            
        print(f"📁 Loaded custom dataset: {X.shape[0]} samples, {X.shape[1]} features")
        
    elif dataset_name == 'breast_cancer':
        data = datasets.load_breast_cancer()
        X, y = data.data, data.target
        task_type = 'classification'
        num_classes = 2
        print(f"📁 Wisconsin Breast Cancer: {X.shape[0]} samples, {X.shape[1]} features")
        
    elif dataset_name == 'digits':
        data = datasets.load_digits()
        X, y = data.data, data.target
        task_type = 'classification'
        num_classes = 10
        print(f"📁 Digits Dataset: {X.shape[0]} samples, {X.shape[1]} features, 10 classes")
        
    elif dataset_name == 'wine':
        data = datasets.load_wine()
        X, y = data.data, data.target
        task_type = 'classification'
        num_classes = 3
        print(f"📁 Wine Dataset: {X.shape[0]} samples, {X.shape[1]} features, 3 classes")
        
    elif dataset_name == 'iris':
        data = datasets.load_iris()
        X, y = data.data, data.target
        task_type = 'classification'
        num_classes = 3
        print(f"📁 Iris Dataset: {X.shape[0]} samples, {X.shape[1]} features, 3 classes")
        
    elif dataset_name == 'diabetes':
        data = datasets.load_diabetes()
        X, y = data.data, data.target
        task_type = 'regression'
        num_classes = 1
        print(f"📁 Diabetes Dataset: {X.shape[0]} samples, {X.shape[1]} features (regression)")
        
    elif dataset_name == 'california_housing':
        data = datasets.fetch_california_housing()
        X, y = data.data, data.target
        task_type = 'regression'
        num_classes = 1
        print(f"📁 California Housing: {X.shape[0]} samples, {X.shape[1]} features (regression)")
        
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")
    
    return X, y, task_type, num_classes

# Load the dataset
X, y, task_type, num_classes = load_dataset(DATASET, CUSTOM_CSV_PATH, TARGET_COLUMN)

# Split into train/val/test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print(f"\n✓ Task Type: {task_type}")
print(f"✓ Train: {X_train.shape[0]} samples")
print(f"✓ Val: {X_val.shape[0]} samples")
print(f"✓ Test: {X_test.shape[0]} samples")
print(f"✓ Features: {X_train.shape[1]}")
if task_type == 'classification':
    print(f"✓ Classes: {num_classes}")

## 4. Create Model

Automatically create a model for your chosen framework:

In [ ]:
input_size = X_train.shape[1]
output_size = num_classes if task_type == 'classification' else 1

if FRAMEWORK == 'pytorch':
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import TensorDataset, DataLoader
    
    class UniversalNet(nn.Module):
        def __init__(self, input_size, output_size, task_type):
            super().__init__()
            hidden1 = min(64, input_size * 4)
            hidden2 = min(32, input_size * 2)
            hidden3 = min(16, input_size)
            
            self.network = nn.Sequential(
                nn.Linear(input_size, hidden1),
                nn.BatchNorm1d(hidden1),
                nn.ReLU(),
                nn.Dropout(0.3),
                
                nn.Linear(hidden1, hidden2),
                nn.BatchNorm1d(hidden2),
                nn.ReLU(),
                nn.Dropout(0.3),
                
                nn.Linear(hidden2, hidden3),
                nn.ReLU(),
                nn.Dropout(0.2),
                
                nn.Linear(hidden3, output_size)
            )
            self.task_type = task_type
        
        def forward(self, x):
            return self.network(x)
    
    print(f"🔨 Created PyTorch model ({input_size} → {output_size})")

elif FRAMEWORK == 'keras':
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    
    def create_keras_model(input_size, output_size, task_type):
        hidden1 = min(64, input_size * 4)
        hidden2 = min(32, input_size * 2)
        hidden3 = min(16, input_size)
        
        model = keras.Sequential([
            layers.Dense(hidden1, activation='relu', input_shape=(input_size,)),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            layers.Dense(hidden2, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            layers.Dense(hidden3, activation='relu'),
            layers.Dropout(0.2),
            
            layers.Dense(output_size, activation='softmax' if task_type == 'classification' else None)
        ])
        return model
    
    print(f"🔨 Created Keras model ({input_size} → {output_size})")

elif FRAMEWORK == 'sklearn':
    from sklearn.neural_network import MLPClassifier, MLPRegressor
    
    hidden1 = min(64, input_size * 4)
    hidden2 = min(32, input_size * 2)
    
    print(f"🔨 Created sklearn model ({input_size} → {output_size})")

print("✅ Model architecture ready!")

## 5. Training Functions

Framework-specific training functions:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Storage for results
history_with_guard = {'train_loss': [], 'val_loss': [], 'train_metric': [], 'val_metric': []}
history_without_guard = {'train_loss': [], 'val_loss': [], 'train_metric': [], 'val_metric': []}

if FRAMEWORK == 'pytorch':
    from overfit_guard.integrations.pytorch import create_pytorch_monitor
    
    def train_pytorch(use_guard=True, verbose=True):
        # Create model
        model = UniversalNet(input_size, output_size, task_type)
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
        
        if task_type == 'classification':
            criterion = nn.CrossEntropyLoss()
            metric_name = 'accuracy'
        else:
            criterion = nn.MSELoss()
            metric_name = 'mse'
        
        # Create monitor
        monitor = None
        if use_guard:
            monitor = create_pytorch_monitor(
                model=model,
                optimizer=optimizer,
                config={'auto_correct': True, 'log_level': 'WARNING'},
                auto_correct=True
            )
        
        # Create data loaders
        train_data = TensorDataset(
            torch.FloatTensor(X_train),
            torch.LongTensor(y_train) if task_type == 'classification' else torch.FloatTensor(y_train)
        )
        val_data = TensorDataset(
            torch.FloatTensor(X_val),
            torch.LongTensor(y_val) if task_type == 'classification' else torch.FloatTensor(y_val)
        )
        
        train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)
        
        history = {'train_loss': [], 'val_loss': [], 'train_metric': [], 'val_metric': []}
        
        for epoch in range(NUM_EPOCHS):
            # Training
            model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0
            
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                
                if task_type == 'classification':
                    loss = criterion(outputs, targets)
                    _, predicted = outputs.max(1)
                    train_correct += predicted.eq(targets).sum().item()
                else:
                    loss = criterion(outputs.squeeze(), targets)
                
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                train_total += targets.size(0)
            
            train_loss /= len(train_loader)
            train_metric = train_correct / train_total if task_type == 'classification' else train_loss
            
            # Validation
            model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0
            
            with torch.no_grad():
                for inputs, targets in val_loader:
                    outputs = model(inputs)
                    
                    if task_type == 'classification':
                        loss = criterion(outputs, targets)
                        _, predicted = outputs.max(1)
                        val_correct += predicted.eq(targets).sum().item()
                    else:
                        loss = criterion(outputs.squeeze(), targets)
                    
                    val_loss += loss.item()
                    val_total += targets.size(0)
            
            val_loss /= len(val_loader)
            val_metric = val_correct / val_total if task_type == 'classification' else val_loss
            
            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['train_metric'].append(train_metric)
            history['val_metric'].append(val_metric)
            
            if verbose and (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Loss: {train_loss:.4f}/{val_loss:.4f} - {metric_name}: {train_metric:.4f}/{val_metric:.4f}")
            
            # Monitor check
            if monitor:
                results = monitor.on_epoch_end(
                    epoch, model,
                    {'loss': train_loss, metric_name: train_metric},
                    {'loss': val_loss, metric_name: val_metric}
                )
                
                if monitor.should_stop:
                    if verbose:
                        print(f"⏹️ Early stopping at epoch {epoch+1}")
                    break
        
        # Test evaluation
        test_data = TensorDataset(
            torch.FloatTensor(X_test),
            torch.LongTensor(y_test) if task_type == 'classification' else torch.FloatTensor(y_test)
        )
        test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)
        
        model.eval()
        test_correct = 0
        test_total = 0
        
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)
                if task_type == 'classification':
                    _, predicted = outputs.max(1)
                    test_correct += predicted.eq(targets).sum().item()
                test_total += targets.size(0)
        
        test_metric = test_correct / test_total if task_type == 'classification' else 0
        
        return history, test_metric, monitor

elif FRAMEWORK == 'keras':
    from overfit_guard.integrations.keras import create_keras_monitor
    
    def train_keras(use_guard=True, verbose=True):
        model = create_keras_model(input_size, output_size, task_type)
        
        if task_type == 'classification':
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            metric_name = 'accuracy'
        else:
            model.compile(optimizer='adam', loss='mse', metrics=['mse'])
            metric_name = 'mse'
        
        callbacks = []
        if use_guard:
            monitor = create_keras_monitor(auto_correct=True, verbose=verbose)
            callbacks.append(monitor)
        
        hist = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=callbacks,
            verbose=1 if verbose else 0
        )
        
        # Convert history
        history = {
            'train_loss': hist.history['loss'],
            'val_loss': hist.history['val_loss'],
            'train_metric': hist.history[metric_name],
            'val_metric': hist.history[f'val_{metric_name}']
        }
        
        # Test
        test_result = model.evaluate(X_test, y_test, verbose=0)
        test_metric = test_result[1]  # Second value is metric
        
        return history, test_metric, monitor if use_guard else None

elif FRAMEWORK == 'sklearn':
    from overfit_guard.integrations.sklearn import create_sklearn_monitor
    from sklearn.metrics import accuracy_score, mean_squared_error
    
    def train_sklearn(use_guard=True, verbose=True):
        if task_type == 'classification':
            model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1, warm_start=True, random_state=42)
            metric_func = accuracy_score
            metric_name = 'accuracy'
        else:
            model = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=1, warm_start=True, random_state=42)
            metric_func = mean_squared_error
            metric_name = 'mse'
        
        monitor = None
        if use_guard:
            monitor = create_sklearn_monitor(
                metric_name=metric_name,
                higher_is_better=(task_type == 'classification')
            )
        
        history = {'train_loss': [], 'val_loss': [], 'train_metric': [], 'val_metric': []}
        
        for epoch in range(NUM_EPOCHS):
            model.fit(X_train, y_train)
            
            train_pred = model.predict(X_train)
            val_pred = model.predict(X_val)
            
            train_metric = metric_func(y_train, train_pred)
            val_metric = metric_func(y_val, val_pred)
            
            train_loss = model.loss_ if hasattr(model, 'loss_') else 0
            val_loss = val_metric if task_type == 'regression' else 1 - val_metric
            
            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['train_metric'].append(train_metric)
            history['val_metric'].append(val_metric)
            
            if verbose and (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{NUM_EPOCHS} - {metric_name}: {train_metric:.4f}/{val_metric:.4f}")
            
            if monitor:
                results = monitor.check_iteration(
                    epoch,
                    {metric_name: train_metric},
                    {metric_name: val_metric},
                    model
                )
                
                if monitor.should_stop:
                    if verbose:
                        print(f"⏹️ Early stopping at epoch {epoch+1}")
                    break
        
        # Test
        test_pred = model.predict(X_test)
        test_metric = metric_func(y_test, test_pred)
        
        return history, test_metric, monitor

print("✅ Training functions ready!")

## 6. Run Training - WITHOUT Guard

Baseline training without overfitting protection:

In [ ]:
print("🚀 Training WITHOUT Overfit Guard...\n")

if FRAMEWORK == 'pytorch':
    history_without_guard, test_without, _ = train_pytorch(use_guard=False)
elif FRAMEWORK == 'keras':
    history_without_guard, test_without, _ = train_keras(use_guard=False)
elif FRAMEWORK == 'sklearn':
    history_without_guard, test_without, _ = train_sklearn(use_guard=False)

print(f"\n✅ Baseline training complete!")
print(f"Test {metric_name}: {test_without:.4f}")

## 7. Run Training - WITH Guard

Training with automatic overfitting detection and correction:

In [ ]:
print("🛡️ Training WITH Overfit Guard...\n")

if FRAMEWORK == 'pytorch':
    history_with_guard, test_with, monitor = train_pytorch(use_guard=True)
elif FRAMEWORK == 'keras':
    history_with_guard, test_with, monitor = train_keras(use_guard=True)
elif FRAMEWORK == 'sklearn':
    history_with_guard, test_with, monitor = train_sklearn(use_guard=True)

print(f"\n✅ Protected training complete!")
print(f"Test {metric_name}: {test_with:.4f}")

if monitor:
    summary = monitor.monitor.get_summary() if hasattr(monitor, 'monitor') else monitor.monitor_obj.get_summary() if hasattr(monitor, 'monitor_obj') else {}
    if summary:
        print(f"\n📊 Overfit Guard Summary:")
        print(f"  - Overfitting detected: {summary.get('overfitting_detected', 0)} times")
        print(f"  - Corrections applied: {summary.get('corrections_applied', 0)}")
        print(f"  - Overfitting rate: {summary.get('overfitting_rate', 0):.1%}")

## 8. Visualize Results

Compare training with and without protection:

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(f'{DATASET.upper()} Dataset - {FRAMEWORK.upper()} Training Comparison', fontsize=16, fontweight='bold')

# WITHOUT Guard - Loss
axes[0, 0].plot(history_without_guard['train_loss'], label='Train Loss', linewidth=2)
axes[0, 0].plot(history_without_guard['val_loss'], label='Val Loss', linewidth=2)
axes[0, 0].set_title('WITHOUT Overfit Guard - Loss', fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# WITHOUT Guard - Metric
axes[0, 1].plot(history_without_guard['train_metric'], label=f'Train {metric_name}', linewidth=2)
axes[0, 1].plot(history_without_guard['val_metric'], label=f'Val {metric_name}', linewidth=2)
axes[0, 1].set_title(f'WITHOUT Overfit Guard - {metric_name.upper()}', fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel(metric_name.upper())
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# WITH Guard - Loss
axes[1, 0].plot(history_with_guard['train_loss'], label='Train Loss', linewidth=2, color='green')
axes[1, 0].plot(history_with_guard['val_loss'], label='Val Loss', linewidth=2, color='orange')
axes[1, 0].set_title('WITH Overfit Guard - Loss', fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# WITH Guard - Metric
axes[1, 1].plot(history_with_guard['train_metric'], label=f'Train {metric_name}', linewidth=2, color='green')
axes[1, 1].plot(history_with_guard['val_metric'], label=f'Val {metric_name}', linewidth=2, color='orange')
axes[1, 1].set_title(f'WITH Overfit Guard - {metric_name.upper()}', fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel(metric_name.upper())
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{DATASET}_{FRAMEWORK}_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Plot saved as: {DATASET}_{FRAMEWORK}_comparison.png")

## 9. Final Comparison

Summary of results:

In [ ]:
print("\n" + "="*60)
print(f"FINAL RESULTS - {DATASET.upper()} on {FRAMEWORK.upper()}")
print("="*60)

# Calculate gaps
final_gap_without = abs(history_without_guard['train_metric'][-1] - history_without_guard['val_metric'][-1])
final_gap_with = abs(history_with_guard['train_metric'][-1] - history_with_guard['val_metric'][-1])
gap_reduction = ((final_gap_without - final_gap_with) / final_gap_without * 100) if final_gap_without > 0 else 0

print(f"\n📊 WITHOUT Overfit Guard:")
print(f"  Test {metric_name}: {test_without:.4f}")
print(f"  Train-Val Gap: {final_gap_without:.4f}")
print(f"  Epochs Trained: {len(history_without_guard['train_loss'])}")

print(f"\n🛡️ WITH Overfit Guard:")
print(f"  Test {metric_name}: {test_with:.4f}")
print(f"  Train-Val Gap: {final_gap_with:.4f}")
print(f"  Epochs Trained: {len(history_with_guard['train_loss'])}")

print(f"\n📈 Improvement:")
metric_change = test_with - test_without
print(f"  Test {metric_name} Change: {metric_change:+.4f} ({metric_change/test_without*100:+.2f}%)")
print(f"  Gap Reduction: {gap_reduction:.1f}%")

if monitor:
    print(f"\n🔍 Detection Details:")
    print(f"  Overfitting Events: {summary.get('overfitting_detected', 0)}")
    print(f"  Auto-Corrections: {summary.get('corrections_applied', 0)}")
    print(f"  Detection Rate: {summary.get('overfitting_rate', 0):.1%}")

print("\n" + "="*60)

## 10. Key Takeaways

**What just happened?**

1. Loaded dataset and split into train/val/test
2. Trained baseline model WITHOUT protection
3. Trained protected model WITH Overfit Guard
4. Compared performance metrics

**Overfit Guard automatically:**
- Monitors train-validation gap every epoch
- Detects overfitting patterns in real-time
- Applies corrections (regularization, learning rate adjustment)
- Triggers early stopping when appropriate

**Try it with your own data:**
1. Set `DATASET = 'custom'`
2. Upload your CSV file
3. Specify target column name
4. Run all cells!

---

**Ready to use Overfit Guard in production?**

```bash
pip install overfit-guard
```

Check out the [GitHub repository](https://github.com/Core-Creates/overfit-guard) for more examples and documentation!

## 11. Key Takeaways

**What just happened?**

1. Loaded dataset and split into train/val/test
2. Trained baseline model WITHOUT protection
3. Trained protected model WITH Overfit Guard
4. Compared performance metrics
5. **NEW:** Generated professional reports in multiple styles!

**Overfit Guard automatically:**
- Monitors train-validation gap every epoch
- Detects overfitting patterns in real-time
- Applies corrections (regularization, learning rate adjustment)
- Triggers early stopping when appropriate
- **NEW:** Generates publication-ready reports for any use case!

**Professional Reporting Features:**
- ✅ **Research Style:** Clean, precise, ready for academic papers
- ✅ **Marketing Style:** Executive-friendly with ROI analysis
- ✅ **Debug Style:** Detailed diagnostics for troubleshooting
- ✅ **Multi-Format Export:** JSON, CSV, HTML, Markdown, LaTeX
- ✅ **Model Cards:** Compliance-ready documentation

**Try it with your own data:**
1. Set `DATASET = 'custom'`
2. Upload your CSV file
3. Specify target column name
4. Run all cells!
5. Generate professional reports for your use case!

---

**Ready to use Overfit Guard in production?**

```bash
pip install overfit-guard
```

**Learn more:**
- 📖 [Documentation](https://github.com/Core-Creates/overfit-guard)
- 💼 [Enterprise Solutions](https://github.com/Core-Creates/overfit-guard/blob/main/ENTERPRISE_BUSINESS_STRATEGY.md)
- 📊 [Analysis Reports](https://github.com/Core-Creates/overfit-guard/blob/main/ANALYSIS_REPORT.md)
- 🚀 [Get Started](https://github.com/Core-Creates/overfit-guard#quick-start)

---

**Upgrade to Professional tier for:**
- Advanced reporting suite (LaTeX, PDF exports)
- Cloud integration & dashboards
- Priority support
- Commercial license

Starting at $49/month → [Learn More](https://github.com/Core-Creates/overfit-guard/blob/main/ENTERPRISE_BUSINESS_STRATEGY.md)

In [ ]:
# Export to multiple formats
from overfit_guard.reporting import ReportExporter

exporter = ReportExporter()

# Export to JSON (for APIs)
exporter.to_json(summary, 'results.json', pretty=True)
print("✅ Exported to results.json")

# Export to CSV (for spreadsheets)
exporter.to_csv(summary, 'results.csv')
print("✅ Exported to results.csv")

# Generate Markdown table
md_table = exporter.to_markdown_table(summary)
print("\n📊 Markdown Table:")
print(md_table)

# Generate HTML table
html_table = exporter.to_html_table(summary)
print("\n✅ HTML table generated (view in results.html)")

print("\n🎉 Professional reporting complete!")
print("\nUse cases:")
print("- Research: Use 'research' style for papers, include LaTeX table generation")
print("- Marketing: Use 'marketing' style for stakeholders, ROI analysis")
print("- Debug: Use 'debug' style for troubleshooting, detailed logs")

### Export Results
Save results in multiple formats for sharing and analysis:

In [ ]:
# Debug style - for troubleshooting
print_overfit_guard_summary(summary, style="debug", include_recommendations=False)

### Debug Style Report
For troubleshooting and internal diagnostics - raw structured data:

In [ ]:
# Marketing style - for executives and stakeholders
print_overfit_guard_summary(summary, style="marketing", include_recommendations=True)

### Marketing Style Report
For executives, stakeholders, and demos - story-driven with emojis:

In [ ]:
# Research style - for academic papers
print_overfit_guard_summary(summary, style="research", include_recommendations=True)

### Research Style Report
For academic papers and technical documentation - clean, precise, no emojis:

In [ ]:
# Import professional reporting module
from overfit_guard.reporting import (
    compute_overfit_guard_summary,
    print_overfit_guard_summary
)

# Compute comprehensive summary
summary = compute_overfit_guard_summary(
    history_baseline=history_without_guard,
    history_guard=history_with_guard,
    test_metric_baseline=test_without,
    test_metric_guard=test_with,
    monitor=monitor,
    metric_name=metric_name,
    higher_is_better=(task_type == 'classification')
)

print("\n" + "="*80)
print("📊 PROFESSIONAL REPORTING DEMO")
print("="*80)

## 10. Professional Reporting (NEW!)

**Generate publication-ready reports in multiple styles:**
- **Research Style:** For academic papers, clean and precise
- **Marketing Style:** For executives and stakeholders, with emojis and narrative
- **Debug Style:** For troubleshooting, raw structured data

This feature is perfect for research papers, product demos, and internal debugging!